In [51]:
import os
# Here you set your data path
path = os.path.expanduser('~/.ros/tracing/lingao_base')

In [52]:
import os
import sys
# Add paths to tracetools_analysis and tracetools_read, assuming a workspace with:
#   src/tracetools_analysis/
#   src/ros-tracing/ros2_tracing/tracetools_read/
dir = '.' # os.path.dirname(os.path.realpath(__file__))
sys.path.insert(0, os.path.join(dir, '../../../tracetools_analysis/tracetools_analysis'))
sys.path.insert(0, os.path.join(dir, '../../../ros-tracing/ros2_tracing/tracetools_read'))
import datetime as dt
from typing import List, Optional
from typing import Tuple
from typing import Union

from bokeh.plotting import figure
# from bokeh.plotting import Figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import Segment
import numpy as np
import pandas as pd

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.cpu_time import CpuTimeDataModelUtil
from tracetools_analysis.processor.cpu_time import CpuTimeHandler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [53]:
# Process
events = load_file(path)
handler = Ros2Handler.process(events)
cpu_handler = CpuTimeHandler()
handler.data.print_data()

converting trace directory: /home/u/.ros/tracing/lingao_base
converted 7654 events in 415 ms
output written to: /home/u/.ros/tracing/lingao_base/converted
 [99%] [Ros2Handler]
====================ROS 2 DATA MODEL===================
Contexts:
                          timestamp    pid version
context_handle                                    
94173828592496  1703843733147876566  15271   4.1.1

Nodes:
                          timestamp    tid      rmw_handle              name namespace
node_handle                                                                           
94173828611568  1703843733155602492  15271  94173828764192  lingao_base_node         /

Publishers (rmw):
                            timestamp                                                                                        gid
publisher_handle                                                                                                                
94173828766640    1703843733155589110   [1, 16, 5, 244, 236

In [54]:
data_util = Ros2DataModelUtil(handler.data)
cpu_util = CpuTimeDataModelUtil(cpu_handler.data)

callback_symbols = data_util.get_callback_symbols()
print("CALLBACK SYMBOLS ================")
print(str(callback_symbols))

output_notebook()

CALLBACK SYMBOLS ================
{94173829181440: 'void (BaseDriver::?)()', 94173829175712: 'void (BaseDriver::?)()', 94173829182720: 'void (BaseDriver::?)()', 94173829178896: 'void (BaseDriver::?)()', 94173829180144: 'void (BaseDriver::?)()', 94173829030584: 'rclcpp::TimeSource::NodeState::attachNode(std::shared_ptr<rclcpp::node_interfaces::NodeBaseInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeTopicsInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeGraphInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeServicesInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeLoggingInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeClockInterface>,std::shared_ptr<rclcpp::node_interfaces::NodeParametersInterface>)::{lambda(std::shared_ptr<rcl_interfaces::msg::ParameterEventconst>)#1}'}


Loading BokehJS ...

In [55]:
# Functions to analyze the pre-processed data

def get_handle(handle_type: str, name: str) -> int:
    if handle_type == 'pub':
        pub_handles = data_util.data.rcl_publishers.loc[
            data_util.data.rcl_publishers['topic_name'] == name
        ].index.values.astype(int)
        # For this demo, we don't expect more than 1 publisher per topic
        assert 1 == len(pub_handles)
        return pub_handles[0]
    if handle_type == 'sub':
        sub_handles = data_util.data.rcl_subscriptions.loc[
            data_util.data.rcl_subscriptions['topic_name'] == name
        ].index.values.astype(int)
        # For this demo, we don't expect more than 1 subscription per topic
        assert 1 == len(sub_handles), f'len={len(sub_handles)}'
        return sub_handles[0]
    if handle_type == 'node':
        node_handles = data_util.data.nodes.loc[
            data_util.data.nodes['name'] == name
        ].index.values.astype(int)
        assert 1 == len(node_handles), f'len={len(node_handles)}'
        return node_handles[0]
    assert False, 'unknown handle_type value'


def get_pub_sub_creation_time(handle_type: str, topic_name: str) -> pd.Timestamp:
    # Get handle
    handle = get_handle(handle_type, topic_name)
    
    df = None
    if handle_type == 'pub':
        # Get timestamp from rcl_publisher_init
        df = data_util.convert_time_columns(data_util.data.rcl_publishers, [], ['timestamp'], False)
    elif handle_type == 'sub':
        # Get timestamp from rcl_subscription_init
        df = data_util.convert_time_columns(data_util.data.rcl_subscriptions, [], ['timestamp'], False)
    else:
        assert False, 'unknown handle_type value'
    return df.loc[handle, 'timestamp']


def get_timer_creation_time(node_name: str) -> pd.Timestamp:
    # Get handle
    node_handle = get_handle('node', node_name)
    
    # Get timer handle from timer-node links
    timer_node_links = data_util.data.timer_node_links.loc[
        data_util.data.timer_node_links['node_handle'] == node_handle
    ].index.values.astype(int)
    # assert 1 == len(timer_node_links), f'len={len(timer_node_links)}'
    timer_handle = timer_node_links[0]
    
    # Get creation timestamp
    df = data_util.convert_time_columns(data_util.data.timers, [], ['timestamp'], False)
    return df.loc[timer_handle, 'timestamp']


def get_timer_callback_ranges(timer_node_name: str, index: int) -> List[Tuple[pd.Timestamp, pd.Timestamp, pd.Timedelta]]:
    # Get timer object
    objs_and_owners = {
        obj: data_util.get_callback_owner_info(obj)
        for obj, _ in callback_symbols.items()
    }
    timer_objs = [
        obj for obj, owner_info in objs_and_owners.items()
        if timer_node_name in owner_info and 'Timer' in owner_info
    ]
    print(timer_objs)
    # assert 1 == len(timer_objs), f'len={len(timer_objs)}'
    timer_obj = timer_objs[index]

    # Get callback durations
    callback_durations = data_util.get_callback_durations(timer_obj)

    # Convert to simple list of tuples
    ranges = []
    for _, row in callback_durations.iterrows():
        begin = row['timestamp']
        duration = row['duration']
        ranges.append((begin, begin + duration, duration))
    return ranges


def get_sub_callback_ranges(sub_topic_name: str) -> List[Tuple[pd.Timestamp, pd.Timestamp, pd.Timedelta]]:
    # Get callback object
    objs_and_owners = {
        obj: data_util.get_callback_owner_info(obj)
        for obj, _ in callback_symbols.items()
    }
    sub_objs = [
        obj for obj, owner_info in objs_and_owners.items()
        if sub_topic_name in owner_info
    ]
    assert 1 == len(sub_objs), f'len={len(sub_objs)}'
    sub_obj = sub_objs[0]

    # Get callback durations
    callback_durations = data_util.get_callback_durations(sub_obj)

    # Convert to simple list of tuples
    ranges = []
    for _, row in callback_durations.iterrows():
        begin = row['timestamp']
        duration = row['duration']
        ranges.append((begin, begin + duration, duration))
    return ranges


def get_publish_times(pub_topic_name: str) -> List[pd.Timestamp]:
    # Get all publish instances
    pub_instances = data_util.get_publish_instances()

    # Get publisher handle
    pub_handle = get_handle('pub', pub_topic_name)

    # Since publish calls go rclcpp->rcl->rmw and since we
    # only know the publisher handle at the rcl level, we first
    # get the indexes of all rcl_publish calls for our publisher
    rcl_pub_indexes = pub_instances.loc[
        pub_instances['publisher_handle'] == pub_handle
    ].index.values.astype(int)
    rcl_pub_indexes = list(rcl_pub_indexes)
    max_index = pub_instances.index.values.astype(int).max()
    # Then we group rclcpp & rmw calls (before & after, respectively) with matching message pointers
    rclcpp_rmw_pub_timestamps = []
    for rcl_pub_index in rcl_pub_indexes:
        # Get message pointer value
        message = pub_instances.iloc[rcl_pub_index]['message']
        # Get corresponding rclcpp_publish row
        rclcpp_timestamp = None
        rclcpp_pub_index = rcl_pub_index - 1
        while rclcpp_pub_index >= 0:
            # Find the first row above with the same message
            row = pub_instances.iloc[rclcpp_pub_index]
            if message == row['message'] and 'rclcpp' == row['layer']:
                rclcpp_timestamp = row['timestamp']
                break
            rclcpp_pub_index -= 1
        # Get corresponding rmw_publish row
        rmw_timestamp = None
        rmw_pub_index = rcl_pub_index + 1
        while rmw_pub_index <= max_index:
            # Find the first row below rcl_publish row with the same message
            row = pub_instances.iloc[rmw_pub_index]
            if message == row['message'] and 'rmw' == row['layer']:
                rmw_timestamp = row['timestamp']
                break
            rmw_pub_index += 1

        assert rclcpp_timestamp is not None and rmw_timestamp is not None
        rclcpp_rmw_pub_timestamps.append(rclcpp_timestamp + (rmw_timestamp - rclcpp_timestamp) / 2)

    return rclcpp_rmw_pub_timestamps

In [56]:
# Functions to display data

def add_durations_to_figure(
    figure: figure,
    segment_type: str,
    durations: List[Union[Tuple[dt.datetime, dt.datetime]]],
    color: str,
    line_width: int = 60,
    legend_label: Optional[str] = None,
) -> None:
    for duration in durations:
        duration_begin, duration_end, _ = duration
        base_kwargs = dict()
        if legend_label:
            base_kwargs['legend_label'] = legend_label
        figure.line(
            x=[duration_begin, duration_end],
            y=[segment_type, segment_type],
            color=color,
            line_width=line_width,
            **base_kwargs,
        )
            

def add_markers_to_figure(
    figure: figure,
    segment_type: str,
    times: List[dt.datetime],
    color: str,
    line_width: int = 60,
    legend_label: Optional[str] = None,
    size: int = 30,
    marker_type: str = 'diamond',
) -> None:
    for time in times:
        base_kwargs = dict()
        if legend_label:
            base_kwargs['legend_label'] = legend_label
        if marker_type == 'diamond':
            figure.diamond(
                x=[time],
                y=[segment_type],
                fill_color=color,
                line_color=color,
                size=size,
                **base_kwargs,
            )
        elif marker_type == 'plus':
            figure.plus(
                x=[time],
                y=[segment_type],
                fill_color=color,
                line_color=color,
                size=size,
                **base_kwargs,
            )
        else:
            assert False, 'invalid marker_type value'

In [63]:
# Analyze and display

creation_timer_lingao = get_timer_creation_time('lingao_base_node')
creation_pub_odom = get_pub_sub_creation_time('pub', '/odom')
ranges_timer_lingao_base = get_timer_callback_ranges('lingao_base_node',3)
times_pub_odom = get_publish_times('/odom')
creation_pub_imu = get_pub_sub_creation_time('pub', '/imu/data_raw')
times_pub_imu = get_publish_times('/imu/data_raw')
creation_pub_battery = get_pub_sub_creation_time('pub', '/battery_state')
times_pub_battery = get_publish_times('/battery_state')
ranges_timer_battery = get_timer_callback_ranges('lingao_base_node',2)
ranges_timer_cmd_vel = get_timer_callback_ranges('lingao_base_node',1)

# For some reason it seems to be displayed in the reverse order on the Y axis
segment_types = [
    'callback cmd_vel',
    'publish /odom',
    'callback timer mainloop',
    'publish /imu/data_raw',
    'publish /battery_state',
    'callback timer battery',
]
start_time = ranges_timer_lingao_base[0][0].strftime('%Y-%m-%d %H:%M')
fig = figure(
    title='lingao_base_node',
    x_axis_label=f'time (from {start_time})',
    y_range=segment_types,
    width=1400,
    height=600,
)
fig.title.align = 'center'
fig.title.text_font_size = '40px'
fig.xaxis[0].formatter = DatetimeTickFormatter()
fig.xaxis[0].axis_label_text_font_size = '30px'
fig.yaxis[0].major_label_text_font_size = '25px'


add_markers_to_figure(fig, 'callback timer lingao_base_node', [creation_timer_lingao], 'blue', marker_type='plus')
add_markers_to_figure(fig, 'publish /odom', [creation_pub_odom], 'blue', marker_type='plus')
add_durations_to_figure(fig, 'callback timer mainloop', ranges_timer_lingao_base, 'blue')
add_markers_to_figure(fig, 'publish /odom', times_pub_odom, 'blue', legend_label='odom node')
add_markers_to_figure(fig, 'publish /imu/data_raw', [creation_pub_imu], 'dodgerblue', marker_type='plus')
add_markers_to_figure(fig, 'publish /imu/data_raw', times_pub_imu, 'dodgerblue', legend_label='imu node')
add_markers_to_figure(fig, 'publish /battery_state', [creation_pub_battery], 'green', marker_type='plus')
add_markers_to_figure(fig, 'publish /battery_state', times_pub_battery, 'green', legend_label='battery node')
add_durations_to_figure(fig, 'callback timer battery', ranges_timer_battery, 'green')
add_durations_to_figure(fig, 'callback cmd_vel', ranges_timer_cmd_vel, 'red')

show(fig)

/opt/ros/humble/lib/python3.10/site-packages/tracetools_analysis/utils/__init__.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_ns_to_datetime] = df[columns_ns_to_datetime].applymap(
/opt/ros/humble/lib/python3.10/site-packages/tracetools_analysis/utils/__init__.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_ns_to_datetime] = df[columns_ns_to_datetime].applymap(
/opt/ros/humble/lib/python3.10/site-packages/tracetools_analysis/utils/__init__.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_ns_to_datetime] = df[columns_ns_to_datetime].applymap(
/opt/ros/humble/lib/python3.10/site-packages/tracetools_analysis/utils/__init__.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_ns_to_datetime] = df[columns_ns_to_datetime].applymap(
/opt/ros/humble/lib/python3.10/site-packages/traceto

[94173829181440, 94173829175712, 94173829182720, 94173829178896, 94173829180144]


/opt/ros/humble/lib/python3.10/site-packages/tracetools_analysis/utils/__init__.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_ns_to_datetime] = df[columns_ns_to_datetime].applymap(
/opt/ros/humble/lib/python3.10/site-packages/tracetools_analysis/utils/__init__.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_ns_to_datetime] = df[columns_ns_to_datetime].applymap(


[94173829181440, 94173829175712, 94173829182720, 94173829178896, 94173829180144]
[94173829181440, 94173829175712, 94173829182720, 94173829178896, 94173829180144]


In [58]:
callback_symbols = data_util.get_callback_symbols()
data_util.get_publish_instances()
callback_object, callback_symbol = list(callback_symbols.items())[4]
callback_durations = data_util.get_callback_durations(callback_object)
# time_per_thread = cpu_util.get_time_per_thread()

# Display, e.g., with bokeh, matplotlib, print, etc.
print(callback_symbol)
print(callback_durations)

get_timer_callback_ranges('lingao_base_node', 2)
# print(time_per_thread)

void (BaseDriver::?)()
                        timestamp                  duration
27  2023-12-29 09:55:33.201142297 0 days 00:00:00.000000390
31  2023-12-29 09:55:33.231162089 0 days 00:00:00.000000433
35  2023-12-29 09:55:33.261194506 0 days 00:00:00.000000330
40  2023-12-29 09:55:33.291134033 0 days 00:00:00.000000602
44  2023-12-29 09:55:33.321819053 0 days 00:00:00.000001225
..                            ...                       ...
632 2023-12-29 09:55:37.371764877 0 days 00:00:00.000001241
637 2023-12-29 09:55:37.401687810 0 days 00:00:00.000001309
641 2023-12-29 09:55:37.431754711 0 days 00:00:00.000001305
645 2023-12-29 09:55:37.461701159 0 days 00:00:00.000001328
650 2023-12-29 09:55:37.491571674 0 days 00:00:00.000001237

[144 rows x 2 columns]
[94173829181440, 94173829175712, 94173829182720, 94173829178896, 94173829180144]


/opt/ros/humble/lib/python3.10/site-packages/tracetools_analysis/utils/__init__.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_ns_to_datetime] = df[columns_ns_to_datetime].applymap(


[(Timestamp('2023-12-29 09:55:34.171569032'),
  Timestamp('2023-12-29 09:55:34.174750195'),
  Timedelta('0 days 00:00:00.003181163')),
 (Timestamp('2023-12-29 09:55:35.171595670'),
  Timestamp('2023-12-29 09:55:35.174765469'),
  Timedelta('0 days 00:00:00.003169799')),
 (Timestamp('2023-12-29 09:55:36.171567988'),
  Timestamp('2023-12-29 09:55:36.174916913'),
  Timedelta('0 days 00:00:00.003348925')),
 (Timestamp('2023-12-29 09:55:37.171225506'),
  Timestamp('2023-12-29 09:55:37.174454563'),
  Timedelta('0 days 00:00:00.003229057'))]

In [59]:
callback_object, callback_symbol = list(callback_symbols.items())[4]
print(callback_object)
print(callback_symbol)

94173829180144
void (BaseDriver::?)()
